# Refactored Script: MultiView/MultiWavelength

In [1]:
#imports
import shdom
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir('/Users/jesserl2/Documents/Code/aviad_pyshdom_dev/pyshdom_dev')

In [2]:
#define the medium

#load cloud.
#locate the 'origin' of the cloud at 0.0,0.0
cloud_scatterer = shdom.grid.load_from_csv('./synthetic_cloud_fields/jpl_les/rico32x37x26.txt', 
                                           density='lwc',origin=(0.0,0.0))

#load atmosphere file for rayleigh. (and eventually gases)
#'Altitude' coordinate is renamed to 'z'.
atmosphere = xr.open_dataset('./ancillary_data/AFGL_summer_mid_lat.nc').rename(Altitude='z')

#extract a chosen temperature_profile and the surface_pressure.
#only model atmosphere below 20 km.
reduced_atmosphere = atmosphere.sel({'z': atmosphere.coords['z'].data[atmosphere.coords['z'].data <= 20.0]})

# -----  make the RTE grid ---------------------------
#RTE grid needs to be made first so that bounding_box for sensors is defined.

#make RTE grid just using cloud_scatterer for horizontal grid and 'merged' z coordinates.
merged_z_coordinate = shdom.grid.combine_z_coordinates([reduced_atmosphere,cloud_scatterer])

#simple 'union' horizontal grid merging for 3D and 1D needs to be fixed.
rte_grid = shdom.grid.make_grid(cloud_scatterer.x.data.min(),cloud_scatterer.x.data.max(),cloud_scatterer.x.data.size,
                           cloud_scatterer.y.data.min(),cloud_scatterer.y.data.max(),cloud_scatterer.y.data.size,
                           merged_z_coordinate)

#this should be done elsewhere.
cloud_scatterer_on_rte_grid = shdom.grid.resample_onto_grid(rte_grid, cloud_scatterer)

#rayleigh doesn't generalize like other scattering objects as it doesn't have a density.
#compute_rayleigh_extinction is also only calculated on 1D grid so it can't be resampled onto the RTE grid
#until it is an optical scatterer.
atmosphere_on_rte_grid = reduced_atmosphere.interp({'z':rte_grid.z})


In [3]:
#define sensors

#This is modified by the user as needed.

#idealized monochromatic orthographic sensors at different wavelengths.
#9 'MISR-like' VIS cameras
#1  'MODIS-like' nadir multi-spectral sensor.

sensor_list = []

#make bounding box for orthographic sensors from RTE grid. This should be internal to the orthographic sensor.
bounding_box = xr.DataArray(
    data=np.array([rte_grid.x.data.min(),rte_grid.y.data.min(),
                             rte_grid.z.data.min(),
                             rte_grid.x.data.max(),
                           rte_grid.y.data.max(),rte_grid.z.data.max()]),
    dims='bbox',
    coords = {'bbox': np.array(['xmin','ymin','zmin', 'xmax','ymax','zmax'])}
)
#add MISR-like sensors
sensor_zenith_list = [70.6,60.0,45.6,26.1,26.1,45.6,60.0,70.6]
sensor_azimuth_list = [90,90,90,90,-90,-90,-90,-90]

for zenith,azimuth in sensor_list:
    sensor_list.append(
        shdom.sensor.orthographic_projection(0.66, bounding_box,0.02,0.02, azimuth, zenith,
                                             altitude='TOA', stokes='I'
                                            )
    
    )

#add MODIS-like sensors
wavelength_list = [0.66, 1.65,2.17]
for wavelength in wavelength_list:
    sensor_list.append(
        shdom.sensor.orthographic_projection(wavelength,bounding_box,0.02,0.02,0.0,0.0,
                                            altitude='TOA',
                                            stokes='I'
                                            )
    )


In [4]:
#find the RTE solvers that are needed.
#(this is relatively simple in this case as only monochromatic RTE solutions are needed)
#This doesn't need modification from case to case for these simple monochromatic sensors.

#num_stokes should be set to choose whether to use num_stokes=USER_SPECIFIED
#even if only radiance needs to be simulated for accuracy reasons.
num_stokes_override_flag = False
num_stokes_override=3

#extract all unique_wavelengths
#this treats even very slightly different wavelengths as unique.
wavelengths = np.unique([sensor.wavelength for sensor in sensor_list])

num_stokes_list = []
nested_sensor_list = []

for wavelength in wavelengths:
    sensor_list_wavelength = []
    for sensor in sensor_list:
        if sensor.wavelength == wavelength:
            if sensor.stokes[3]:
                min_stokes = 4
            elif sensor.stokes[1] or sensor.stokes[2]:
                min_stokes = 3
            elif sensor.stokes[0]:
                min_stokes = 1
            if (num_stokes_override_flag) & (min_stokes < num_stokes_override):
                min_stokes = num_stokes_override
            num_stokes_list.append(min_stokes)
            
            sensor_list_wavelength.append(sensor)
    nested_sensor_list.append(sensor_list_wavelength)
                

In [5]:
#create optical properties for the necessary wavelengths.


#There is also a need to map from the 'scatterers' defined earlier to the number of particle types needed here
#we could have several Aerosol mixtures and also liquid water.
#need to be careful.

cloud_mie_mono_list = []
for wavelength in wavelengths:
    #just calculates the mie_tables - may take a while.
    print('calculating mie tables: this may take a while')
    mie_mono_table = shdom.mie.get_mono_table('Water',(wavelength,wavelength)) 
    cloud_mie_mono_list.append(mie_mono_table)

#users make choices here.
#Here we model the cloud as gamma distribution. It is not currently specified in the loaded cloud.
#An attribute 'distribution_type' could be added to load_from_csv/load_from_netcdf initialized to None
#if not supplied.

#The choice of integration radii varies in each mie_table as it is sensitive to the size parameter
#and therefore a 'size distribution' must be made for each wavelength.
cloud_size_distribution_list =  [shdom.size_distribution.get_size_distribution_grid(
                                                            mie_mono_table.radius.data,
    size_distribution_function=shdom.size_distribution.gamma,particle_density=1.0,
    reff=[4.0,25.0,25,'logarithmic','micron'],
    veff=[0.09,0.11,2,'linear','unitless'],                                                                               
) for mie_mono_table in cloud_mie_mono_list] #veff in table cannot be one number else there are nans in optical property interpolation.

cloud_poly_table_list = [shdom.mie.get_poly_table(cloud_size_distribution,mie_mono_table) for mie_mono_table,cloud_size_distribution
                         in zip(cloud_mie_mono_list, cloud_size_distribution_list)]

#define the missing variable. Ideally this would be done based on a choice at the beginning when
#the scatterer is loaded along with the choice about a distribution shape.
cloud_scatterer_on_rte_grid['veff'] = (cloud_scatterer_on_rte_grid.reff.dims, np.full_like(cloud_scatterer_on_rte_grid.reff.data, fill_value=0.1))

cloud_optical_scatterers = [shdom.medium.table_to_grid(cloud_scatterer_on_rte_grid, poly_table) for poly_table in cloud_poly_table_list]



calculating mie tables: this may take a while
calculating mie tables: this may take a while
calculating mie tables: this may take a while


In [6]:
#rayleigh was not finished so I added this.
#this function and rayleigh.py need to be integrated and slimmed down.
def clumsy_get_rayleigh(wavelengths, atmosphere_on_rte_grid):
    rayleigh_poly_tables = shdom.rayleigh.compute_table(wavelengths).rename('legcoef')
    rayleigh_extinction = shdom.rayleigh.compute_extinction(wavelengths,atmosphere_on_rte_grid.Temperature,
                                                           surface_pressure=atmosphere_on_rte_grid.Pressure.data[0])

    rayleigh_ssalb = xr.DataArray(
                        name='ssalb',
                        data = np.ones(rayleigh_extinction.extinction.shape,dtype=np.float32),
                        dims=['wavelength','z']
    )

    rayleigh_table_index = xr.DataArray(
                        name='table_index',
                        data = np.ones(rayleigh_extinction.extinction.shape, dtype=np.int),
                        dims=['wavelength','z']
    )

    rayleigh = xr.merge([rayleigh_extinction,rayleigh_ssalb, rayleigh_table_index]).broadcast_like(rte_grid)

    rayleigh = rayleigh.set_coords('table_index')
    rayleigh_final = xr.merge([rayleigh, rayleigh_poly_tables.expand_dims(dim='table_index', axis=-2)])
    return rayleigh_final

rayleigh = clumsy_get_rayleigh(wavelengths,atmosphere_on_rte_grid)
rayleigh_scatterer_list = []
for wavelength in wavelengths:
    temp = rayleigh.sel({'wavelength':wavelength})
    temp.attrs['wavelength_center'] = wavelength #RTE checks that all scatterers have the same wavelength
    #based on a wavelength_center attribute so this must exist.
    rayleigh_scatterer_list.append(temp)


In [7]:
#choose other inputs for each RTE. In this case it is entirely determined by wavelength.
#but the surface parameterization may also want to be chosen depending on num_stokes for example.
#For simplicity I use all the same ones.

name_list = [None for wavelength in wavelengths]

surface_list = [shdom.surface.fixed_lambertian_surface(0.01) for wavelength in wavelengths]

source_list = [shdom.source.solar_source(145.0,0.0,solarflux=1.0) for wavelength in wavelengths]

numerical_params_list = [shdom.configuration.get_config('./default_config.json') for wavelength in wavelengths]

#combine media for each RTE.
medium_list = []
for rayleigh, optical in zip(rayleigh_scatterer_list,cloud_optical_scatterers):
    medium_list.append([optical, rayleigh])
    
    
    

In [8]:
#make solver list
solver_list = []
for numerical_params,medium,source,surface,num_stokes,name in zip(numerical_params_list,
                                                              medium_list,
                                                              source_list,
                                                              surface_list,
                                                              num_stokes_list,
                                                              name_list):
    
    solver = shdom.solver.RTE(numerical_params,medium,source,surface,num_stokes=num_stokes,name=name)
    solver_list.append(solver)

In [ ]:
#solver list can be distributed in parallel or in serial for solution
#the solved solvers are modified inplace and can be found in solver_list.

for solver in solver_list:
    solver.solve(maxiter=100)

In [ ]:
#sensors are still unfinished so I just render one image.
output,sensor_with_image = solver_list[0].integrate_to_sensor(nested_sensor_list[0][0])

In [ ]:
plt.figure()
plt.imshow(sensor_with_image.I.data.reshape(sensor_with_image.image_shape.data,order='F'))
plt.colorbar()
plt.show()